# Author: Faolán Hamilton
## Project Description: Analyse the planning applications submitted to Clare County Council over the past year 
### Source - https://planning.geohive.ie/datasets/housinggovie::irishplanningapplications/explore?layer=0&showTable=true

# Strategy
- Filter the dataset to only Clare County Council (>500,000 records down to 11,000)
- Filter the dataset to only the past year (2025 inclusive ~ 1,000)
- Filter out unneccessary headings 

## Clean-up
- Eircode out of address using regex
- Can I use commas  to determine a townland for address and group by settlement?

# Potential Questions:
- How many of the planning applications are granted?
- Determine a type for each (Residential, commercial etc.)
- Number of residential units 

In [1]:
#Modules to import 

import datetime as dt

import pandas as pd

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

## Step 1: I went to the irish planning applications geohive website (https://planning.geohive.ie/datasets/housinggovie::irishplanningapplications/explore?filters=eyJQbGFubmluZ0F1dGhvcml0eSI6WyJDbGFyZSBDb3VudHkgQ291bmNpbCJdLCJSZWNlaXZlZERhdGUiOlsxNzM1Njg5NjAwMDAwLDI3MDQyMzM2MDAwMDBdfQ%3D%3D&layer=0&showTable=true) and filtered the +500,000 results to only show Clare County Council applications since 1/1/2025.

### Note - I repeated this step in the New Year to get a list from all of 2025

## Step 2: Download the results as a csv from the webiste.

## Step 3 - Upload the file to this project folder

In [ ]:
df = pd.read_csv("ClareCoCo_PlanningApps_2025.csv")

In [ ]:
headers = df.columns.tolist()
headers

In [ ]:
#remove the headers that are not relevant
drop_col_list = ['OBJECTID','Planning Authority','ITMEasting','ITMNorthing','Applicant Forename','Applicant Surname','Applicant Address','WithdrawnDate','GrantDate','ExpiryDate','Appeal Reference Number','Link Application Details','AppealSubmittedDate','FIRequestDate','FIRecDate', 'Appeal Status', 'Site Id', 'ETL_DATE', 'ORIG_FID']
df.drop(columns=drop_col_list, inplace=True)
df.head(1)

In [ ]:
df['ReceivedDate']

In [ ]:
#https://stackoverflow.com/questions/38067704/how-to-change-the-datetime-format-in-pandas
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
#https://sqlpey.com/python/solved-how-to-change-the-datetime-format-in-pandas/
received_dateformat = pd.to_datetime(df.ReceivedDate)
received_dateformat.dt.strftime('%d/%m/%Y')

In [ ]:
x = df['NumResidentialUnits']
plt.hist(x)
plt.show()